In [18]:
import tweepy
import nltk
import pandas as pd
import pickle
import numpy as np 
import re 
import os
import random
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.classify import ClassifierI
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import style
import time
from statistics import mode
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [19]:
class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers

    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
            
        return mode(votes)

    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf

In [20]:
# Open dataset 
df_neg = pd.read_csv('../dataset/n_out.csv')
df_pos = pd.read_csv('../dataset/p_out.csv')
# class = 0 = negative, 1 = positive\
dt_transformed = pd.concat([df_pos, df_neg])
dt_transformed


,tweets,label
0,@caprichOreality Fica assim não miga &lt;3 Tud...,1.0
1,Amanhã é dia de dar um trato na palestra para ...,1.0
2,@thankovsky @patorebaichado eu também tenho :)...,1.0
3,@JoseAbrantes6 Prefiro amar o meu clube nas vi...,1.0
4,@Bel_Reedus Recomendamos que vá até uma loja C...,1.0
...,...,...
5242,só começando a temporada e já com muito lixo n...,0.0
5243,que brotp de caráter bicho :( https://t.co/9LF...,0.0
5244,@PriAlcantara Preciso dessa determinação :(,0.0
5245,@macedoflora to sem :(,0.0


In [21]:
all_words = [] 
documents = [] 

def processing_text_to_pos_tag(row_table):
    all_stopwords = stopwords.words('portuguese')
    t_data = re.sub("@[0-9A-za-z:]+", "", row_table[0])  # removing @user
    t_data = re.sub("https?://[A-Za-z0-9./&#]+", "", t_data)  # removing http links
    t_data = re.sub("(:..)", "", t_data)  # removing faces
    t_data = t_data.lower()
    t_data = t_data.split()
    ps = PorterStemmer()
    t_data = [ps.stem(word) for word in t_data if not word in set(all_stopwords) if len(word) > 2]
    t_data = ' '.join(t_data)
    
    return row_table[1], t_data

In [22]:
# allowed_word_types = ["J"]

for row in dt_transformed.values:
    c,t = processing_text_to_pos_tag(row)
    documents.append( (t,c) )
    words = word_tokenize(t)
#   pos_t = nltk.pos_tag(words)
    for w in words:
        all_words.append(w)
        
save_documents = open("../pickled_algos/documents.pickle","wb")
pickle.dump(documents, save_documents)
save_documents.close()

all_words = nltk.FreqDist(all_words)
word_features = list(all_words.keys())

save_word_features = open("../pickled_algos/word_features5k.pickle","wb")
pickle.dump(word_features, save_word_features)
save_word_features.close()

In [23]:
def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

In [24]:
featuresets = [(find_features(rev), category) for (rev, category) in documents]
random.shuffle(featuresets)

In [25]:
testing_set = featuresets[4000:]
training_set = featuresets[:4000]

In [26]:
classifier = nltk.NaiveBayesClassifier.train(training_set)
print("Original Naive Bayes Algo accuracy percent:", (nltk.classify.accuracy(classifier, testing_set))*100)
# classifier.show_most_informative_features(15)

save_classifier = open("../pickled_algos/originalnaivebayes5k.pickle","wb")
pickle.dump(classifier, save_classifier)
save_classifier.close()

MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("MNB_classifier accuracy percent:", (nltk.classify.accuracy(MNB_classifier, testing_set))*100)

save_classifier = open("../pickled_algos/MNB_classifier5k.pickle","wb")
pickle.dump(MNB_classifier, save_classifier)
save_classifier.close()

BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)
print("BernoulliNB_classifier accuracy percent:", (nltk.classify.accuracy(BernoulliNB_classifier, testing_set))*100)

save_classifier = open("../pickled_algos/BernoulliNB_classifier5k.pickle","wb")
pickle.dump(BernoulliNB_classifier, save_classifier)
save_classifier.close()

LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogisticRegression_classifier, testing_set))*100)

save_classifier = open("../pickled_algos/LogisticRegression_classifier5k.pickle","wb")
pickle.dump(LogisticRegression_classifier, save_classifier)
save_classifier.close()

LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
print("LinearSVC_classifier accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)

save_classifier = open("../pickled_algos/LinearSVC_classifier5k.pickle","wb")
pickle.dump(LinearSVC_classifier, save_classifier)
save_classifier.close()

SGDC_classifier = SklearnClassifier(SGDClassifier())
SGDC_classifier.train(training_set)
print("SGDClassifier accuracy percent:",nltk.classify.accuracy(SGDC_classifier, testing_set)*100)

save_classifier = open("../pickled_algos/SGDC_classifier5k.pickle","wb")
pickle.dump(SGDC_classifier, save_classifier)
save_classifier.close()

Original Naive Bayes Algo accuracy percent: 70.4304772383441
MNB_classifier accuracy percent: 71.53073855040573
BernoulliNB_classifier accuracy percent: 71.39320588639802
LogisticRegression_classifier accuracy percent: 71.68202448081419
LinearSVC_classifier accuracy percent: 69.46774859029019
SGDClassifier accuracy percent: 69.57777472149635


In [5]:
documents_f = open("../pickled_algos/documents.pickle", "rb")
documents = pickle.load(documents_f)
documents_f.close()


word_features5k_f = open("../pickled_algos/word_features5k.pickle", "rb")
word_features = pickle.load(word_features5k_f)
word_features5k_f.close()




In [6]:
open_file = open("../pickled_algos/originalnaivebayes5k.pickle", "rb")
classifier = pickle.load(open_file)
open_file.close()


open_file = open("../pickled_algos/MNB_classifier5k.pickle", "rb")
MNB_classifier = pickle.load(open_file)
open_file.close()



open_file = open("../pickled_algos/BernoulliNB_classifier5k.pickle", "rb")
BernoulliNB_classifier = pickle.load(open_file)
open_file.close()


open_file = open("../pickled_algos/LogisticRegression_classifier5k.pickle", "rb")
LogisticRegression_classifier = pickle.load(open_file)
open_file.close()


open_file = open("../pickled_algos/LinearSVC_classifier5k.pickle", "rb")
LinearSVC_classifier = pickle.load(open_file)
open_file.close()


open_file = open("../pickled_algos/SGDC_classifier5k.pickle", "rb")
SGDC_classifier = pickle.load(open_file)
open_file.close()

In [27]:
voted_classifier = VoteClassifier(classifier,
                                  MNB_classifier,
                                  BernoulliNB_classifier,
                                  LogisticRegression_classifier,
                                  SGDC_classifier)

In [38]:
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import json


def out(text):
    feats = find_features(text)
    return voted_classifier.classify(feats),voted_classifier.confidence(feats)


ckey=""
csecret=""\
atoken=""
asecret=""


class listener(StreamListener):

    def on_data(self, data):
        
        all_data = json.loads(data)
        self.tweet = all_data["text"]
        self.id = all_data["id"]
        self.sentiment_value, self.confidence = out(self.tweet)
        
        print(self.tweet, self.sentiment_value, self.confidence)

        if self.confidence*100 >= 80:
            output = open("twitter-out.txt","a")
            output.write(str(self.sentiment_value))
            output.write('\n')
            output.close()
        else:
            output = open("twitter-out.txt","a")
            output.write(str(2))
            output.write('\n')
            output.close()
            
        return True

    def on_error(self, status):
        print(status)

auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

twitterStream = Stream(auth, listener())
twitterStream.filter(track=["bolsonaro"])

RT @mitoboIsonaro: Eu não sei vocês, mas eu continuo fechado com o Bolsonaro. #BolsonaroHeroiDoBrasil 0.0 1.0
RT @viniciusduarte: "Bolsonaro não entrega o que prometeu"

o programa de governo do Bolsonaro parecia ter sido escrito por um ancap de 14… 1.0 1.0
RT @RubensNunesMBL: Flávio Bolsonaro não esperou nem dar uma semana da decisão do STJ que praticamente livrou sua cara e já foi comprar uma… 0.0 1.0
RT @MarceloFreixo: Flávio Bolsonaro segue investindo em imóveis... Agora ele  comprou uma mansão de R$ 6 milhões em Brasília. Por uma dessa… 1.0 1.0
RT @carapanarana: Exemplo de superação! Senador humildão fica milionário com loja de chocolates. 1.0 1.0
RT @Crissrc: O cúmulo do absurdo é votar no PT e querer ser fiscal de honestidade do governo Bolsonaro🙄 0.0 1.0
RT @carmelonetobr: Boa notícia! Governo Bolsonaro enviará ao Ceará até quarta-feira (3), 122 mil doses de vacina contra covid-19. 1.0 1.0
RT @coimbraricardo: A mansão de 6 milhões do Flávio Bolsonaro é um salão de depilação a l

@ChicoPascoal @xicosa Follow the money. Melhorou? de qualquer maneira claro que não é do Moro, mas ele usou bastant… https://t.co/mjKFnhIGF6 1.0 0.8
RT @DCM_online: Padre chama Bolsonaro de ‘imoral’ e ‘genocida’ por gestão na pandemia: “Tem o prazer de matar” https://t.co/hs7iz5ixUf 1.0 1.0
RT @sensacionalista: Mansão de R$ 6 milhões comprada por Flávio Bolsonaro terá que passar por reforma pois está cheia de rachadinhas https:… 1.0 1.0
Olha o Brasil de Bolsonaro entrando pra história mundial. 1.0 0.8
RT @blogdojosias: Ao dizer que não errou, Bolsonaro erra outra vez https://t.co/DUfjfLPINj via @UOLNoticias @UOL 1.0 0.8
Onde vamos parar? Os bandidos que enriqueceram com dinheiro da milícia, da contravenção estão ganhando a batalha
https://t.co/vZ17LxAxjN 1.0 1.0
RT @JoaoBoscoDeMor3: SÉRGIO MORO condenou LULA porque ele é ladrão. Mas ñ foi para eleger Bolsonaro, ele e o delta delanhol, queriam eleger… 0.0 1.0
RT @BelinhaPoodleBR: Estão criticando o Flávio Bolsonaro por uma mansão de R$5

@henriolliveira_ @jairbolsonaro Esse Flávio Dino nunca tinha visto falar.Bolsonaro até 2026! 1.0 1.0
A @policiafederal fez o que com essa sujeita que disse que ia dar uma facada no Presidente Jair Messias Bolsonaro?? https://t.co/3MbumdDnUL 0.0 1.0
RT @marilizpj: Bolsonaro quer o voto dos ignorantes e dos extremistas, diz André Trigueiro, no EstudioI, sobre a estratégia para 2022. Bing… 1.0 0.8
RT @ThiagoResiste: Enquanto dezenas de milhares de pessoas estão intubadas à beira da morte por covid e mais de 1500 morrem TODOS OS DIAS,… 1.0 1.0
RT @albertocantalic: Vídeo mostra a super mansão comprada por Flavio Bolsonaro com dinheiro não declarado https://t.co/8FRXl8mZz9 1.0 1.0
RT @filipebarrost: A forma como Moro saiu do MJ, após interferir ilegalmente na PF impedindo o PR de exercer suas atribuições, tem relação… 1.0 1.0
RT @opropriofaka: Esse pode ser o golpe 

https://t.co/QKFkz6Ii27 1.0 0.8
RT @MidiaNINJA: Enquanto aumenta a fome no país e milhares de brasileiros sofrem com aumentos 

RT @em_com: Romeu Zema não assina nota de governadores contra Jair Bolsonaro

https://t.co/H8ozIGiaJc 0.0 0.6
RT @heldersalomao: Esse é o Brasil de Bolsonaro!

Em Vitória-ES, tem carne até com alarme antifurto no comércio. https://t.co/FNCIJ0vnPX 1.0 1.0
Heroi é Flavio Bolsonaro que com uma franquia de chocolate conseguiu comprar uma mansão de 6 milhões.
Uns nascem co… https://t.co/uj3LIThAD2 1.0 0.8
RT @marinalang: Da Tijuca para uma mansão de R$ 6 milhões. Realmente, a política é uma carreira lucrativa. 1.0 1.0
RT @Anb_Li: Um país que precisa driblar seu próprio presidente para sobreviver. Não seria mais fácil retirá-lo? 1.0 0.8
RT @mila_sayuri: Em vez de assinarem nota contra o PR Bolsonaro, prestem contas dos valores recebidos para o combate da pandemia. 

É o mín… 1.0 1.0
@taoquei1 Isso é referente a nota de repúdio dos governadores pelo bolsonaro ter divulgado quanto repassou para cad… https://t.co/tbMF5mcE3g 0.0 0.8
RT @filipebarrost: A forma como Moro saiu do MJ, após interferi

RT @kimpaim: Isso me incomoda porque eu gosto do Bolsonaro

Mas não se enganem, isso é uma cortina de fumaça, eu ainda não esqueci do depoi… 0.0 1.0


KeyboardInterrupt: 